## View SED Galaxies in one HP
- author : Sylvie Dagoret-Campagne
- afflilation : IJCLab/IN2P3/CNRS
- creation date : 2025-03-10
- last update : 2025-03-10
- nersc python KERNEL : desc-python, or desc-python-bleed

In [ ]:
import galsim
import os
import pandas as pd
import h5py

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

props = dict(boxstyle='round', facecolor="white", alpha=0.1)


import matplotlib.colors as colors
import matplotlib.cm as cmx

In [ ]:
plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
datadir = "/global/cfs/cdirs/descssim/imSim/skyCatalogs_v1.1.2"
throughputs_dir = "/global/cfs/cdirs/descssim/imSim/lsst/data/throughputs_aug_2021"

In [ ]:
os.environ["THROUGHPUTS_DIR"] = throughputs_dir

### Select one HP 

In [ ]:
hpnum = 10552
data_name = f"skyCatalogs_v1.1.2 hp={hpnum}"

In [ ]:
fn_sed = f"galaxy_sed_{hpnum}.hdf5"
fn_galaxy = f"galaxy_{hpnum}.parquet"
fn_flux = f"galaxy_flux_{hpnum}.parquet"

In [ ]:
ffn_sed = os.path.join(datadir,fn_sed)
if not os.path.exists(ffn_sed):
    raise Exception(f"File {ffn_sed} DOES NOT exists")
else:
    with h5py.File(ffn_sed, "r") as f:
        print("all keys = ",f.keys())
        meta = f["meta"]
        print(meta)
        a_group_key0 = list(f.keys())[0]
        a_group_key1 = list(f.keys())[1]
        print("a_group_key0 =  = ",a_group_key0)
        print("a_group_key1 =  = ",a_group_key1)
        data0 = list(f[a_group_key0])
        data1 = list(f[a_group_key1])
        print("data0 = ",data0[0])
        print("data1 = ",data1[0])

In [ ]:
ffn_galaxy = os.path.join(datadir,fn_galaxy)
if not os.path.exists(ffn_sed):
    raise Exception(f"File {ffn_galaxy} DOES NOT exists")
else:
    df_g = pd.read_parquet(ffn_galaxy)

In [ ]:
df_g .head()

In [ ]:
len(df_g)

In [ ]:
fig, ax = plt.subplots(1,1)
df_g["redshift"].hist(bins=100,ax=ax)
ax.set_xlabel("redshift")
ax.set_title(data_name)

In [ ]:
ffn_flux = os.path.join(datadir,fn_flux)
if not os.path.exists(ffn_flux):
    raise Exception(f"File {ffn_fluxy} DOES NOT exists")
else:
    df_f = pd.read_parquet(ffn_flux)

In [ ]:
df_f .head()

In [ ]:
len(df_f)

In [ ]:
#with h5py.File(ffn_sed, 'r') as file:
    # Function to recursively print the HDF5 dataset hierarchy
#    def print_hdf5_item(name, obj):
        # name is in path format like /group1/group2/dataset
#        if isinstance(obj, h5py.Group):
            # Do something like creating a dictionary entry
#            print(f'Group: {name}')
        #elif isinstance(obj, h5py.Dataset):
        #    # Do something with obj like converting to a pandas.Series 
            # and storing to a dictionary entry
        #    print(f'Dataset: {name}')
        #    break;

    # Visit all items in the HDF5 file and print their names
#    file.visititems(print_hdf5_item)


In [ ]:
ffn_sed = os.path.join(datadir,fn_sed)
if not os.path.exists(ffn_sed):
    raise Exception(f"File {ffn_sed} DOES NOT exists")

f = h5py.File(ffn_sed, "r")

group_wl = f["meta"]
wl = group_wl["wave_list"][()]

group_gal = f["galaxy"]

In [ ]:
print("subgroups - galaxies : ",group_gal.keys())

In [ ]:
list_of_subgroups_gal = list(group_gal.keys())
for idx_sg, subgroup_name in enumerate(list_of_subgroups_gal):
    print(f"{idx_sg}::{subgroup_name}")
    subgroup = group_gal[subgroup_name]
    list_of_ids_gal = list(subgroup.keys())
    for idx_gal,gal_num in enumerate(list_of_ids_gal):
        print(f"\t {idx_gal}::{gal_num}")
        the_dataset = subgroup[gal_num][()]
        break
    break

In [ ]:
the_colors = ["r","b","g"]
the_labels = ["disk", "bulge", "knot" ] 
Units = "fnu"

fig,ax = plt.subplots(1,1,figsize=(12,6))
for idx in range(3):
    ax.plot(wl/10,the_dataset[idx,:],color=the_colors[idx],label=the_labels[idx],lw=3)
ax.legend() 
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel(Units)
ax.set_yscale("log")
ax.set_xscale("log")
plt.show()

In [ ]:
group_number = 105520000
gal_number = 10552000000000

In [ ]:
gal_number//group_number

In [ ]:
def retrievedatafromspectrum(ffn_sed,number_gal):
    """
    Parameters
    - ffn_sed : fullfinename
    """
    
    group_number = number_gal // 100_000

    if not os.path.exists(ffn_sed):
        raise Exception(f"File {ffn_sed} DOES NOT exists")

    try:
        f = h5py.File(ffn_sed, "r")
    except:
        raise Exception(f"File {ffn_sed} failed to be open as h5 file by h5py")

    # top level groups

    ## group providing wavelength
    try:
        group_wl = f["meta"]
        wl = group_wl["wave_list"][()]
    except:
        raise Exception(f"File {ffn_sed} :: h5py cannot retrieve meta/wave_list dataset")

    ## group containing all galaxies data
    try:
        group_gal = f["galaxy"]
    except:
        raise Exception(f"File {ffn_sed} :: h5py cannot retrieve galaxy group")

    ## subgroup indexed by group_number
    try:
        subgroup_name = f"{group_number}"
        subgroup = group_gal[subgroup_name]
    except:
        raise Exception(f"File {ffn_sed} :: h5py cannot retrieve galaxy/{subgroup_name} subgroup")

    ## subgroup of the galaxy
    subgroup_gal_name = f"{number_gal}"
    try:
        subsubgroup = subgroup[subgroup_gal_name]
        ## retrieve the fnu fluxes (3 arrays for disk, bulge and knot)
        dataset = subsubgroup[()]
    except:
        raise Exception(f"File {ffn_sed} :: h5py cannot retrieve galaxy/{subgroup_name}/{subgroup_gal_name} dataset")
    
    return wl,dataset

In [ ]:
wl, fnusarr = retrievedatafromspectrum(ffn_sed ,gal_number)

In [ ]:
title = f"{ffn_sed} : \n {gal_number}"

In [ ]:
the_colors = ["r","b","g"]
the_labels = ["disk", "bulge", "knot" ] 
Units = "fnu"

fig,ax = plt.subplots(1,1,figsize=(12,6))
for idx in range(3):
    ax.plot(wl/10,fnusarr [idx,:],color=the_colors[idx],label=the_labels[idx],lw=3)
ax.legend() 
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel(Units)
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_title(title)
plt.show()